# Guess " the Word" Game
# using talking replace typing for search
#AI---definitely this is one AI area 
1. converting audio to words
2. remove the noise
3. Guess word game

In [1]:
#pip install SpeechRecognition
#checking if it is available
import speech_recognition as sr
print(sr.__version__)

3.8.1


In [13]:
r = sr.Recognizer()

In [15]:
#r.recognize_google()

PyPI中有一些现成的语音识别软件包。其中包括：

•apiai

•google-cloud-speech

•pocketsphinx

•SpeechRcognition

•watson-developer-cloud

•wit

SpeechRecognition 的核心就是识别器类。


Recognizer API 主要目是识别语音，每个 API 都有多种设置和功能来识别音频源的语音，分别是：

recognize_bing(): Microsoft Bing Speech

recognize_google(): Google Web Speech API

recognize_google_cloud(): Google Cloud Speech - requires installation of the google-cloud-speech package

recognize_houndify(): Houndify by SoundHound

recognize_ibm(): IBM Speech to Text

recognize_sphinx(): CMU Sphinx - requires installing PocketSphinx

recognize_wit(): Wit.ai

以上七个中只有 recognition_sphinx（）可与CMU Sphinx 引擎脱机工作， 其他六个都需要连接互联网。

SpeechRecognition 目前支持的文件类型有：

WAV: 必须是 PCM/LPCM 格式

AIFF

AIFF-C

FLAC: 必须是初始 FLAC 格式；OGG-FLAC 格式不可用

In [16]:
harvard = sr.AudioFile('harvard.wav')


In [18]:
with harvard as source:
     audio = r.record(source) 

In [19]:
type(audio)

speech_recognition.AudioData

In [20]:
r.recognize_google(audio)

'the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle taste fine with ham tacos al Pastore are my favorite a zestful food is the hot cross bun'

# play the file to verify the content
If you’re wondering where the phrases in the “harvard.wav” file come from, they are examples of Harvard Sentences. These phrases were published by the IEEE in 1965 for use in speech intelligibility testing of telephone lines. They are still used in VoIP and cellular testing today.

The Harvard Sentences are comprised of 72 lists of ten phrases. You can find freely available recordings of these phrases on the Open Speech Repository website. Recordings are available in English, Mandarin Chinese, French, and Hindi. They provide an excellent source of free material for testing your code.

In [22]:
with harvard as source:
    audio1 = r.record(source, duration=4)
    audio2 = r.record(source, duration=4)   

In [23]:
r.recognize_google(audio1)

'the stale smell of old beer lingers'

In [24]:
r.recognize_google(audio2)

'it takes heat to bring out the odor a cold dip'

#The offset and duration keyword arguments are useful for segmenting an audio file if you have prior knowledge of the structure of the speech in the file

In [26]:
with harvard as source:
    audio = r.record(source, offset=4, duration=3)
r.recognize_google(audio)

'it takes heat to bring out the odor'

Noise is a fact of life. All audio recordings have some degree of noise in them, and un-handled noise can wreck the accuracy of speech recognition apps.
try to play the file then see how to handle nosie

In [29]:
jackhammer = sr.AudioFile('jackhammer.wav')

In [30]:
with jackhammer as source:
    audio = r.record(source)
r.recognize_google(audio)

'the snail smell of old beer drinkers'

In [31]:
with jackhammer as source:
    r.adjust_for_ambient_noise(source)
    audio = r.record(source)
r.recognize_google(audio)

'still smell old gear vendors'

The adjust_for_ambient_noise() method reads the first second of the file stream and calibrates the recognizer to the noise level of the audio. Hence, that portion of the stream is consumed before you call record() to capture the data.

In [32]:
with jackhammer as source:
    r.adjust_for_ambient_noise(source, duration=0.5)
    audio = r.record(source)
r.recognize_google(audio)

'the snail smell like old beermongers'

When working with noisy files, it can be helpful to see the actual API response. Most APIs return a JSON string containing many possible transcriptions. The recognize_google() method will always return the most likely transcription unless you force it to give you the full response.You can do this by setting the show_all keyword argument of the recognize_google() method to True

In [33]:
r.recognize_google(audio, show_all=True)

{'alternative': [{'confidence': 0.67151636,
   'transcript': 'the snail smell like old beermongers'},
  {'transcript': 'the still smell like old beermongers'},
  {'transcript': 'the snail smell like old beer mongers'},
  {'transcript': 'the still smell like old beer mongers'},
  {'transcript': 'the still smell like old beer vendors'}],
 'final': True}

# Working With Microphones

In [48]:
#get a list of microphone names
sr.Microphone.list_microphone_names()

['Microsoft Sound Mapper - Input',
 'Microphone (Logitech USB Headse',
 'Microphone (Realtek High Defini',
 'Microsoft Sound Mapper - Output',
 'Speakers (Logitech USB Headset ',
 'Speaker/HP (Realtek High Defini']

In [51]:
import pyaudio

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
#for each audio device, determine if is an input or an output and add it to the appropriate list and dictionary
for i in range (0,numdevices):
        if p.get_device_info_by_host_api_device_index(0,i).get('maxInputChannels')>0:
                print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))

        if p.get_device_info_by_host_api_device_index(0,i).get('maxOutputChannels')>0:
                print("Output Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))

devinfo = p.get_device_info_by_index(1)
print ("Selected device is ",devinfo.get('name'))
if p.is_format_supported(44100.0,  # Sample rate
                         input_device=devinfo["index"],
                         input_channels=devinfo['maxInputChannels'],
                         input_format=pyaudio.paInt16):
  print ('Yay!')
p.terminate()

Input Device id  0  -  Microsoft Sound Mapper - Input
Input Device id  1  -  Microphone (Logitech USB Headse
Input Device id  2  -  Microphone (Realtek High Defini
Output Device id  3  -  Microsoft Sound Mapper - Output
Output Device id  4  -  Speakers (Logitech USB Headset 
Output Device id  5  -  Speaker/HP (Realtek High Defini
Selected device is  Microphone (Logitech USB Headse
Yay!


In [55]:
#pip install pyaudio
mic = sr.Microphone(device_index=1)


In [56]:
with mic as source:
    audio88 = r.listen(source)

In [57]:
r.recognize_google(audio88)

'hello hello hello hello hello hello'

In [58]:
#suppose you are in a noising office
with mic as source:
    r.adjust_for_ambient_noise(source)
    audio = r.listen(source)

import random
import time

import speech_recognition as sr


def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.

    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response


if __name__ == "__main__":
    # set the list of words, maxnumber of guesses, and prompt limit
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    # create recognizer and mic instances
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    # get a random word from the list
    word = random.choice(WORDS)

    # format the instructions string
    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    # show instructions and wait 3 seconds before starting the game
    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        # get the guess from the user
        # if a transcription is returned, break out of the loop and
        #     continue
        # if no transcription returned and API request failed, break
        #     loop and continue
        # if API request succeeded but no transcription was returned,
        #     re-prompt the user to say their guess again. Do this up
        #     to PROMPT_LIMIT times
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak!'.format(i+1))
            guess = recognize_speech_from_mic(recognizer, microphone)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("I didn't catch that. What did you say?\n")

        # if there was an error, stop the game
        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        # show the user the transcription
        print("You said: {}".format(guess["transcription"]))

        # determine if guess is correct and if any attempts remain
        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        # determine if the user has won the game
        # if not, repeat the loop if user has more attempts
        # if no attempts left, the user loses the game
        if guess_is_correct:
            print("Correct! You win!".format(word))
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break

In [59]:
import speech_recognition as sr
from guessing_game import recognize_speech_from_mic
r = sr.Recognizer()
m = sr.Microphone()

In [68]:
recognize_speech_from_mic(r, m)

{'error': None, 'success': True, 'transcription': 'Orange'}

Only the following methods accept a language keyword argument:

recognize_bing()
recognize_google()
recognize_google_cloud()
recognize_ibm()
recognize_sphinx()
To find out which language tags are supported by the API you are using, you’ll have to consult the corresponding documentation. 